# GoogLeNet
有没有一种方法既能保持网络结构的稀疏性，又能利用密集矩阵的高计算性能。大量的文献表明可以将稀疏矩阵聚类为较为密集的子矩阵来提高计算性能，就如人类的大脑是可以看做是神经元的重复堆积，因此，GoogLeNet团队提出了Inception网络结构，就是构造一种“基础神经元”结构，来搭建一个稀疏性、高计算性能的网络结构。

## 导入库

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

## 参数

In [ ]:
BATCH_SIZE = 10000
EPOCH = 5

## 数据导入及调整大小

In [ ]:
class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32)/255.0, axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32)/255.0, axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.num_train, self.num_test = self.train_images.shape[0], self.test_images.shape[0]
        
    def get_batch_train(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_images)[0], batch_size)
        resized_images = tf.image.resize_with_pad(self.train_images[index], 224, 224,)
        return resized_images.numpy(), self.train_labels[index]
    
    def get_batch_test(self, batch_size):
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        resized_images = tf.image.resize_with_pad(self.test_images[index], 224, 224,)
        return resized_images.numpy(), self.test_labels[index]


In [ ]:
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(BATCH_SIZE)

## 定义层

In [ ]:
class Inception(keras.layers.Layer):
    def __init__(self,c1, c2, c3, c4):
        super().__init__()
        self.p1_1 = keras.layers.Conv2D(c1, kernel_size=1, activation='relu', padding='same')
        self.p2_1 = keras.layers.Conv2D(c2[0], kernel_size=1, padding='same', activation='relu')
        self.p2_2 = keras.layers.Conv2D(c2[1], kernel_size=3, padding='same', activation='relu')
        self.p3_1 = keras.layers.Conv2D(c3[0], kernel_size=1, padding='same', activation='relu')
        self.p3_2 = keras.layers.Conv2D(c3[1], kernel_size=5, padding='same', activation='relu')
        self.p4_1 = keras.layers.MaxPool2D(pool_size=3, padding='same', strides=1)
        self.p4_2 = keras.layers.Conv2D(c4, kernel_size=1, padding='same', activation='relu')

    def call(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return tf.concat([p1, p2, p3, p4], axis=-1)

## 定义网络

In [ ]:
b1 = keras.models.Sequential()
b1.add(keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', activation='relu'))
b1.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

b2 = keras.models.Sequential()
b2.add(keras.layers.Conv2D(64, kernel_size=1, padding='same', activation='relu'))
b2.add(keras.layers.Conv2D(192, kernel_size=3, padding='same', activation='relu'))
b2.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

b3 = keras.models.Sequential()
b3.add(Inception(64, (96, 128), (16, 32), 32))
b3.add(Inception(128, (128, 192), (32, 96), 64))
b3.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

b4 = keras.models.Sequential()
b4.add(Inception(192, (96, 208), (16, 48), 64))
b4.add(Inception(160, (112, 224), (24, 64), 64))
b4.add(Inception(128, (128, 256), (24, 64), 64))
b4.add(Inception(112, (144, 288), (32, 64), 64))
b4.add(Inception(256, (160, 320), (32, 128), 128))
b4.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

b5 = keras.models.Sequential()
b5.add(Inception(256, (160, 320), (32, 128), 128))
b5.add(Inception(384, (192, 384), (48, 128), 128))
b5.add(keras.layers.GlobalAvgPool2D())

model = keras.models.Sequential([b1, b2, b3, b4, b5, tf.keras.layers.Dense(10)])

## 损失和优化

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## 训练

In [ ]:
num_iter = dataLoader.num_train//BATCH_SIZE
for e in range(EPOCH):
    for n in range(num_iter):
        x_batch, y_batch = dataLoader.get_batch_train(BATCH_SIZE)
        model.fit(x_batch, y_batch)

In [ ]:
x_test, y_test = dataLoader.get_batch_test(2000)
model.evaluate(x_test, y_test, verbose=2)